In [1]:
import ee
ee.Initialize()
from iteru import *
from IPython.display import display
import ipywidgets
import datetime

In [2]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [270]:
aoi = Map.aoi

In [134]:
vis_param = get_vis_params(S2)

In [35]:
demvis = {'min':488,'max':1863,'palette':['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

In [434]:
dataset = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2')\
           .filter(ee.Filter.bounds(aoi))\
           .select('DSM')\
           .median()\
           .clip(aoi)

In [435]:
#Map.add_layer_widgets(dataset,demvis,'GRED_DEM')

In [899]:
S2 = ee.ImageCollection("COPERNICUS/S2_SR")\
     .filter(ee.Filter.bounds(aoi))\
     .filter(ee.Filter.date('2022-1-01','2022-2-1'))\
     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',2))

In [900]:
S2_median = S2.median().clip(aoi)

In [901]:
S2.size().getInfo()

6

In [902]:
#Map.add_layer_widgets(S2_median,vis_param,'S2_Median')

In [903]:
ndwi = S2_median.normalizedDifference(['B3', 'B8'])

In [904]:
ndwiMasked = ndwi.updateMask(ndwi.gte(0))

In [905]:
#Map.add_layer_widgets(ndwiMasked,{'min':0,'max':0.5,'palette': ['00FFFF', '0000FF']},'NDWI')

In [906]:
v = ee.Image(1).updateMask(ndwiMasked).reduceToVectors(
           geometry = ndwiMasked.geometry(),
           crs =  ndwiMasked.projection(),
           scale = 10 , 
           geometryType = 'polygon',
           eightConnected = False,
           labelProperty =  'water_cover',
           bestEffort = True
)

In [907]:
def calc_area(feature):
    area = feature.geometry().area(maxError = 1)
    
    return feature.set({'Area':area})

In [908]:
v = v.map(calc_area)

In [909]:
v = ee.Feature(v.sort('Area', False).first())

In [910]:
#Map.add_layer_widgets(v,{'color':'red'},'Vector')

In [911]:
lake_dem = dataset.clip(v)

In [912]:
Map.add_layer_widgets(lake_dem,demvis,'Lake DEM')

In [913]:
ele = lake_dem.reduceRegion(
                         reducer = ee.Reducer.max(),
                         geometry = lake_dem.geometry(), 
                         scale = 10, 
                        maxPixels = 1e10)

In [914]:
water_level = ele.getInfo()['DSM']

In [915]:
water_level

601

In [916]:
import numpy as np

In [917]:
#latlng = ee.Image.pixelLonLat().addBands(lake_dem)
latlng = lake_dem.reduceRegion(reducer=ee.Reducer.toList(), geometry=v.geometry(), maxPixels=1e11, scale=30)

In [918]:
elev_values = np.array((ee.Array(latlng.get("DSM")).getInfo()),dtype=np.int64)

In [919]:
elev_values.size

392253

In [920]:
volume = water_level - elev_values[elev_values < water_level]

In [921]:
(volume.sum()*900)/1e9

15.8324976